# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [86]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [87]:
data = pd.read_csv("cities.csv")
data


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [88]:
gmaps.configure(api_key=g_key)

locations = data[["Lat", "Lng"]].astype(float)
humidity = data["Humidity"].astype(float)

In [89]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [90]:
##Temperature 
#temp = data['Max Temp'].tolist()
temp = data[data['Max Temp'] <= 79]
temp2 = data[data['Max Temp'] > 70]
perf_temp = pd.merge(temp, temp2)

##Wind Speed
wind = perf_temp[perf_temp['Wind Speed'] < 10]
wind

##Cloudiness
clouds = wind[wind['Cloudiness'] <= 0]
clouds

data_complete = clouds.dropna()
data_complete

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
11,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
14,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
21,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
26,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
43,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
61,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [91]:
hotel_df = pd.DataFrame(data_complete)

hotel_df['Hotel Name'] = ""
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
11,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
14,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
21,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,
26,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,
43,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,
61,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,


In [165]:
pon_target_coordinates = "-20.63, -46.00"
pon_target_search = "ponta do sol"
pon_t_radius = 5000
pon_target_type = "lodging"

params = {"location": pon_target_coordinates,
    "keyword": pon_target_search,
    "radius": pon_t_radius,
    "type": pon_target_type,
    "key": g_key    
}
 

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

responseP = requests.get(base_url, params=params)

places_dataP = responseP.json()
places_dataP


{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}

In [166]:
#t_radius = 5000
#target_coordinates = "32.86, 59.22"
#target_type = "hotels"
#target_search = "birjand"



########

cid_target_coordinates = "-30.17, -50.22"
cid_target_search = "cidreira"
cid_t_radius = 5000
cid_target_type = "hotels"

params = {"location": cid_target_coordinates,
    "keyword": cid_target_search,
    "radius": cid_t_radius,
    "type": cid_target_type,
    "key": g_key    
}
 

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

response = requests.get(base_url, params=params)
response

places_data = response.json()
places_data


{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': -30.1769388,
     'lng': -50.2089142},
    'viewport': {'northeast': {'lat': -30.0395772, 'lng': -50.16861129999999},
     'southwest': {'lat': -30.2364182, 'lng': -50.3946563}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/geocode-71.png',
   'id': 'eb1ec0eb6026a57094416ac4df9fd907b72a8137',
   'name': 'Cidreira',
   'photos': [{'height': 1280,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116704569421211365714">Méri Passos</a>'],
     'photo_reference': 'CmRaAAAAuXRew0Y0old4Yip1w3PLaqRdyaJyT8z8ptIn-BSsZt_N1DYZplKwojodGMmsnTJlGKNwrc3AmwJhS2raKSSXhf1-_TvW85GQ_dO7eUt_h0W6Ll08fyVx8u09sxXlcPaTEhBlbY0InrRcBxmCAmtHYEdaGhQm2kQMf_Amq5yhZxDNB4pEdByTsQ',
     'width': 720}],
   'place_id': 'ChIJSTXVPnYQGJURfH7OTA3IzS0',
   'reference': 'ChIJSTXVPnYQGJURfH7OTA3IzS0',
   'scope': 'GOOGLE',
   'types': ['administrative_area_level_2', 'political'],
   'vicinity': 'Cidreira - RS, Br

In [167]:
vai_t_radius = 5000
vai_target_coordinates = "15.34, 74.49"
vai_target_type = "hotels"
vai_target_search = "lodging"

params = {"location": cid_target_coordinates,
    "keyword": cid_target_search,
    "radius": cid_t_radius,
    "type": cid_target_type,
    "key": g_key    
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

responseV = requests.get(base_url, params=params)

places_dataV = responseV.json()
places_dataV

{'html_attributions': [],
 'results': [{'geometry': {'location': {'lat': -30.1769388,
     'lng': -50.2089142},
    'viewport': {'northeast': {'lat': -30.0395772, 'lng': -50.16861129999999},
     'southwest': {'lat': -30.2364182, 'lng': -50.3946563}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/geocode-71.png',
   'id': 'eb1ec0eb6026a57094416ac4df9fd907b72a8137',
   'name': 'Cidreira',
   'photos': [{'height': 1280,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116704569421211365714">Méri Passos</a>'],
     'photo_reference': 'CmRaAAAAsXISggQmkUUONqJ1mELw-T0dj5eGkhy1Ty7ypMmVvZrgWiIvExcNLOK67VtYnkBxR86CmsaXpcu2HlAbUXGc3_90fQYtuEBpYb-SSrynHdXZsq7zm_TpzkoBC5wA2gSwEhCd8Sd0TyljI_lge_AeD4ZOGhRJe9AU7cRYlJpNf9Zg3OU1dITuOQ',
     'width': 720}],
   'place_id': 'ChIJSTXVPnYQGJURfH7OTA3IzS0',
   'reference': 'ChIJSTXVPnYQGJURfH7OTA3IzS0',
   'scope': 'GOOGLE',
   'types': ['administrative_area_level_2', 'political'],
   'vicinity': 'Cidreira - RS, Br

In [181]:
nad_t_radius = 5000
nad_target_coordinates = "35.17, -2.93"
nad_target_type = "hotels"
nad_target_search = "lodging"

params = {"location": nad_target_coordinates,
    "keyword": nad_target_search,
    "radius": nad_t_radius,
    "type": nad_target_type,
    "key": g_key    
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

responseN = requests.get(base_url, params=params)

places_dataN = responseN.json()
places_dataN

print(f"Closest Nador hotel is {results[0]['name']}.")
hotel_df.loc[index, 'Hotel Name'] = results[0]['name']


Closest Nador hotel is Hotel Marchica Lagoon Resort, Nador Morocco.


In [151]:
#base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"location": target_coordinates,
    "keyword": target_search,
    "radius": t_radius,
    "type": target_type,
    "key": g_key    
}
 
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    city1 = row['City']

    # add keyword to params dict
    params['keyword'] = city1

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city1}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    results


    # assemble url and make API request

        


Retrieving Results for Index 11: ponta do sol.
Retrieving Results for Index 14: cidreira.
Retrieving Results for Index 21: vaini.
Retrieving Results for Index 26: nador.
Retrieving Results for Index 43: mogok.
Retrieving Results for Index 61: birjand.
Retrieving Results for Index 61: birjand.


[]

In [179]:
print(f"Retrieving Results for Index {index}: {city1}.")
response = requests.get(base_url, params=params).json()
    
results = response['results']
results


Retrieving Results for Index 61: birjand.


[{'geometry': {'location': {'lat': 35.2067501, 'lng': -2.9121019},
   'viewport': {'northeast': {'lat': 35.20815157989272,
     'lng': -2.910970770107277},
    'southwest': {'lat': 35.20545192010728, 'lng': -2.913670429892722}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
  'id': '2864908fe3bb02966a8ece18aa8c46c45a214716',
  'name': 'Hotel Marchica Lagoon Resort, Nador Morocco',
  'opening_hours': {'open_now': True},
  'photos': [{'height': 1841,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/105364499116920086873">A Google User</a>'],
    'photo_reference': 'CmRaAAAAmLdu9fodARkgfflZJVP3oJTXmZWdb7lo9hgPtYl15vzDwyga4h6SSMp7ItV3eTeHuT3pEGIOOK4PR6sdlZ1ILbhu8Ql_W2x5Cb9sIp38ZYp25eDVRNtAKOADzONKY9VqEhA-adzHHB2GdMFuXxLdG4SxGhQVX1LAWclEonMM6C_CDCl74tlgHg',
    'width': 2758}],
  'place_id': 'ChIJIy4dBa6hdw0RUPEo0hhgfM4',
  'plus_code': {'compound_code': '634Q+P5 Nador, Morocco',
   'global_code': '8C7V634Q+P5'},
  'rating': 4.7,
  'r

In [134]:
try:
    print(f"Closest {city1} hotel is {results[0]['name']}.")
    hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
except (KeyError, IndexError):
    print("Missing field/result... skipping.")

Missing field/result... skipping.


In [182]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
11,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
14,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
21,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,
26,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,
43,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,
61,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,"Hotel Marchica Lagoon Resort, Nador Morocco"


In [120]:
Birn = hotel_df[hotel_df['Max Temp'] == 71.60] 
Birn

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
61,444,birjand,0,IR,1558378880,26,32.86,59.22,71.6,9.17,Birjand


In [131]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

hotel_info
locations

,Lat,Lng
11,-20.63,-46.00
14,-30.17,-50.22
21,15.34,74.49
26,35.17,-2.93
43,22.92,96.51
61,32.86,59.22


In [ ]:
gmaps.configure(api_key=g_key)

locations = data[["Lat", "Lng"]].astype(float)
humidity = data["Humidity"].astype(float)

In [ ]:
# Add marker layer ontop of heat map


# Display figure
